In [1]:
from bidict import bidict
from collections import defaultdict
import itertools
import os
import pandas as pd
import pickle
import lzma

from conllu import parse, parse_incr
import numpy as np
from cp_orth import orth_als
import sktensor

from decomp_pmi import VerbTensor

import logging
logging.basicConfig(level=logging.DEBUG, format='%(levelname)-8s [%(lineno)d] %(message)s')

# DepCC: salience

In [2]:
svo_count = pd.read_csv('/mnt/store/home/makrai/project/verb-tensor/depCC.tsv', sep='\t')
modes = ['nsubj', 'ROOT', 'dobj']
svo_count, log_total = VerbTensor().append_pmi(svo_count=svo_count, modes=modes)

INFO     [33] Computing marginals..
INFO     [35] Computing 2-marginals..
INFO     [42] Computing Dice..


AttributeError: 'NotImplementedType' object has no attribute 'reindex_like'

In [ ]:
svo_count[svo_count.columns[10:]].describe(percentiles=[])

In [ ]:
svo_count[svo_count.freq>1].sort_values('pmi', ascending=False).head()

In [ ]:
svo_count[svo_count.freq>1].sort_values('iact_info', ascending=False).head()

In [ ]:
svo_count[svo_count.freq>10].sort_values('dice', ascending=False).head()

In [ ]:
svo_count[svo_count.freq>0].sort_values('salience', ascending=False).head()

In [ ]:
svo_count[svo_count.freq>0].sort_values('iact_sali', ascending=False).head()

# Get UMBC

In [ ]:
def get_umbc_dict():
    umbc_dir = '/mnt/store/home/makrai/data/language/english/corp/umbc_WebBase/English/'
    freq = defaultdict(int)
    for filen in os.listdir(umbc_dir):
        logging.info(filen)
        for i, sentence in  enumerate(parse_incr(lzma.open(os.path.join(umbc_dir, filen), mode='rt',
                                                           encoding="utf-8"))):
            if not i % 100000:
                logging.debug(i)
            root = sentence.to_tree()
            subj, obj = '', ''
            for child in root.children:
                if 'subj' in child.token['deprel']:
                    if subj:
                        #logging.warn('subj: {}'.format((subj, child.token['lemma'], sentence)))
                        continue
                    subj = child.token['lemma']
                elif child.token['deprel'] == 'obj':
                    if obj:
                        #logging.warn('obj: {}'.format((obj, child.token['lemma'], sentence)))
                        continue            
                    obj = child.token['lemma']
            #if bool(obj) and bool(subj):
            freq[(subj, root.token['lemma'], obj)] += 1
        #pickle.dump(freq, open('/mnt/store/home/makrai/project/verb-tensor/umbc_freq.pkl', mode='wb'))
    return freq

In [ ]:
def get_umbc_df():
    freq = pickle.load(open('/mnt/store/home/makrai/project/verb-tensor/umbc_freq.pkl', mode='rb'))
    freq_df = pd.DataFrame.from_records(list(freq.items()), columns=['svo', 'freq'])
    freq_df[['subj', 'verb', 'obj']] = pd.DataFrame(freq_df.svo.tolist(), index=freq_df.index)                                                                                                                       
    del freq_df['svo']
    return freq_df

# Mazsola DB

In [ ]:
mazsola = pickle.load(open(
    '/mnt/permanent/Language/Hungarian/Dic/sass15-535k-igei-szerkezet/mazsola_adatbazis.pkl',
    mode='rb'))

In [ ]:
mazsola_df =pd.read_csv(
    '/mnt/permanent/Language/Hungarian/Dic/sass15-535k-igei-szerkezet/mazsola_adatbazis_svo_freq.tsv', sep='\t',
    keep_default_na=False)

In [ ]:
mazsola_df, log_total = append_pmi(mazsola_df, compute_freq=False)

In [ ]:
mazsola_df.sort_values('iact_info').head()

# Top triples

In [ ]:
svo_count.sort_values('freq', ascending=False).head()

In [ ]:
svo_count[svo_count.freq>100].sort_values('pmi', ascending=False).head()

In [ ]:
svo_count[(svo_count.freq>100) & (svo_count.ACC != 'NULL')].sort_values('iact_info', ascending=False).head()

In [ ]:
svo_count[(svo_count.ACC != 'NULL')].sort_values('salience', ascending=False).head()

In [ ]:
svo_count.sort_values('iact_sali', ascending=False).head()

# UMBC

In [ ]:
svo_count, log_total = append_pmi(freq_df, modes=['subj', 'verb', 'obj'], compute_freq=False)#, debug_index=2234759)

In [ ]:
svo_count[(svo_count.subj != '') & (svo_count.obj != '')].sort_values('freq', ascending=False).head()

In [ ]:
svo_count[svo_count.freq>100].sort_values('pmi', ascending=False).head()

In [ ]:
svo_count[(svo_count.freq>100) & (svo_count.subj != '') & (svo_count.obj != '')].sort_values(
    'iact_info', ascending=True).head()

In [ ]:
svo_count[(svo_count.subj != '') & (svo_count.obj != '')].sort_values('salience', ascending=False).head()

In [ ]:
svo_count[(svo_count.freq>100)].sort_values('dice', ascending=False).head()